<a href="https://colab.research.google.com/github/varaslaw/AISingersCoverGen/blob/main/Hina_Mod_AICoverGen_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AICoverGen WebUI (Modded by [Hina](https://discordlookup.com/user/444684887363026974))
Оригинальный репозиторий: [varaslaw](https://github.com/varaslaw/AISingersCoverGen)

Обновлённый колаб: стек Python 3.10–3.12, CUDA 12.x, Gradio 5.23.x. Нажмите `Runtime → Run all` и дождитесь публичного URL. При желании можно поставить опциональные зависимости (Mega/librosa).

In [ ]:
#@title Clone repository
from IPython.display import clear_output
import os, shutil

repo_name = 'AISingersCoverGen'
repo_dir = f'/content/{repo_name}'
repo_url = 'https://github.com/varaslaw/AISingersCoverGen.git'
install_to_drive = False  #@param {type:'boolean'}

if install_to_drive:
    from google.colab import drive
    drive.mount('/content/drive')
    target_dir = f'/content/drive/MyDrive/{repo_name}'
else:
    target_dir = repo_dir

if os.path.exists(target_dir):
    shutil.rmtree(target_dir)
!git clone --depth=1 {repo_url} {target_dir}
%cd {target_dir}
clear_output()
print(f"Cloned repository to {target_dir}")


In [ ]:
#@title Install requirements + download core models
from pathlib import Path
EXTRAS = False  #@param {type:'boolean'}
%cd $repo_dir
!python -m pip install -U pip wheel
has_gpu = Path('/usr/bin/nvidia-smi').exists()
req_file = 'requirements-gpu.txt' if has_gpu else 'requirements.txt'
print(f"Installing dependencies from {req_file} (extras={'on' if EXTRAS else 'off'})")
!python -m pip install -q -r {req_file}
if EXTRAS:
    !python -m pip install -q -r requirements-optional.txt
!sudo apt-get update
!sudo apt-get install -y sox ffmpeg
!python src/download_models.py
print('Finished installing requirements and downloading core models')


In [ ]:
#@title Run WebUI
%cd $repo_dir
Url = "Gradio" #@param ['Gradio','Ngrok','Cloudflared']
Token = "" #@param {type:'string'}
if Url == "Gradio":
    !python src/webui-ru.py --share
elif Url == "Cloudflared":
    !curl -LO https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
    !dpkg -i cloudflared-linux-amd64.deb
    !rm -f nohup.out
    !nohup cloudflared tunnel --url localhost:9999 &
    import time
    time.sleep(5)
    !grep -oE "https://[a-zA-Z0-9.-]+\.trycloudflare\.com" nohup.out
    !python src/webui-ru.py --listen-port 9999
else:
    !python -m pip install -q pyngrok
    from pyngrok import ngrok
    ngrok.set_auth_token(Token)
    ngrok.kill()
    cover_tunnel = ngrok.connect(9999)
    print("CoverGen URL:", cover_tunnel.public_url)
    !python src/webui-ru.py --listen-port 9999


[![](https://i.pinimg.com/474x/de/72/9e/de729ecfa41b69901c42c82fff752414.jpg)](https://discordlookup.com/user/444684887363026974)